In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/PASCAL/'

/content/gdrive/MyDrive/PASCAL


In [ ]:
!wget http://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar
!wget http://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar

In [ ]:
!tar -xvf VOCtrainval_06-Nov-2007.tar
!tar -xvf VOCtest_06-Nov-2007.tar

PASCAL VOC 2007 directory
```
/VOCdevkit//VOC2007/
├── Annotations        
│   └── *.xml
├── ImageSets
│   ├── Layout
│   │   └── *.txt
│   ├── Main
│   │   └── *.txt
│   └── Segmentation
│        └── *.txt
├── JPEGImages
│   └── .jpg
├── SegmentationClass
│   └── *.png
└── SegmentationObject
     └── *.png

```


ImageSets
- class 별 train/val/test txt 파일
- object와 class 일치 여부(일치 : 1, 불일치 : -1) 

Annotations
```
<object>
    <name>dog</name>                # object class
    <pose>Left</pose>               # object의 방향 
    <truncated>1</truncated>        # object 잘림 여부
    <difficult>0</difficult>        # object 인식 난이도
    <bndbox>                        # object bounding box
        <xmin>48</xmin>             # bounding box 좌상단 x 좌표
        <ymin>240</ymin>            # bounding box 좌상단 y 좌표
        <xmax>195</xmax>            # bounding box 우하단 x 좌표 
        <ymax>371</ymax>            # bounding box 우하단 y 좌표 
    </bndbox>
</object>
```

참조 : [PASCAL VOC 2007 데이터셋 다운로드 및 확인 방법](https://ndb796.tistory.com/500)

In [ ]:
!cat './VOCdevkit/VOC2007/Annotations/000001.xml'

<annotation>
	<folder>VOC2007</folder>
	<filename>000001.jpg</filename>
	<source>
		<database>The VOC2007 Database</database>
		<annotation>PASCAL VOC2007</annotation>
		<image>flickr</image>
		<flickrid>341012865</flickrid>
	</source>
	<owner>
		<flickrid>Fried Camels</flickrid>
		<name>Jinky the Fruit Bat</name>
	</owner>
	<size>
		<width>353</width>
		<height>500</height>
		<depth>3</depth>
	</size>
	<segmented>0</segmented>
	<object>
		<name>dog</name>
		<pose>Left</pose>
		<truncated>1</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>48</xmin>
			<ymin>240</ymin>
			<xmax>195</xmax>
			<ymax>371</ymax>
		</bndbox>
	</object>
	<object>
		<name>person</name>
		<pose>Left</pose>
		<truncated>1</truncated>
		<difficult>0</difficult>
		<bndbox>
			<xmin>8</xmin>
			<ymin>12</ymin>
			<xmax>352</xmax>
			<ymax>498</ymax>
		</bndbox>
	</object>
</annotation>


In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
def extract_file(imageset_path, file_type, class_name=""):
    if class_name != "":
        class_name += '_'
    file_path = imageset_path + class_name + file_type + '.txt'

    cls = []

    with open(file_path, 'r') as f:
        reader = f.readlines()
        for line in reader:
            check = line.strip().split(' ')
            if class_name != "":
               if check[-1] == str(1):
                   cls.append(check[0])
            else:           
                cls.append(check[0])
    return np.array(cls)

In [ ]:
classes = set()
dir_name = './VOCdevkit/VOC2007/'

imageset = 'ImageSets/Main/'
imageset_path = os.path.join(dir_name, imageset)

annotation = 'Annotations/'
annotation_path = os.path.join(dir_name, annotation)

jpeg = 'JPEGImages/'
jpeg_path = os.path.join(dir_name, jpeg)

class_dir = os.listdir(imageset_path)
annot_dir = os.listdir(annotation_path)
jpeg_dir = os.listdir(jpeg_path)

In [ ]:
for cls in class_dir:
    cls = cls.split('/')[-1].split('_')[0]
    if '.txt' not in cls:
        classes.add(cls)

print(len(classes), classes)

20 {'boat', 'cow', 'motorbike', 'pottedplant', 'dog', 'car', 'tvmonitor', 'person', 'chair', 'aeroplane', 'bus', 'bicycle', 'bird', 'cat', 'bottle', 'horse', 'sheep', 'sofa', 'diningtable', 'train'}


In [ ]:
samples = {
    'train': extract_file(imageset_path, 'train'),
    'validation': extract_file(imageset_path, 'val'),
    'test': extract_file(imageset_path, 'test')
}

In [ ]:
!pip install xmltodict

In [ ]:
col_names = ['file_name', 'file_type', 'object_length', 'object', 'xmin', 'ymin', 'xmax', 'ymax', 'pose', 'truncated', 'difficult']
df_dict = {col_name : [] for col_name in col_names}

In [ ]:
for obj in value['annotation']['object']:
    print(obj)

OrderedDict([('name', 'person'), ('pose', 'Left'), ('truncated', '0'), ('difficult', '0'), ('bndbox', OrderedDict([('xmin', '185'), ('ymin', '62'), ('xmax', '279'), ('ymax', '199')]))])
OrderedDict([('name', 'horse'), ('pose', 'Left'), ('truncated', '0'), ('difficult', '0'), ('bndbox', OrderedDict([('xmin', '90'), ('ymin', '78'), ('xmax', '403'), ('ymax', '336')]))])


In [ ]:
import xmltodict

def parse_xml(annotation_path, file_name, file_type):
    global df_dict
    xml_name = file_name + '.xml'
    xml_path = os.path.join(annotation_path, xml_name)

    with open(xml_path, 'rb') as f:
        xml_dict = xmltodict.parse(f)

        objects = xml_dict['annotation']['object']

        if isinstance(objects, list):
            for obj in objects:
                df_dict['file_name'].append(xml_dict['annotation']['filename'])
                df_dict['file_type'].append(file_type)
                df_dict['object_length'].append(len(objects))
                df_dict['object'].append(obj['name'])

                bndbox = obj['bndbox']
                df_dict['xmin'].append(int(bndbox['xmin']))
                df_dict['ymin'].append(int(bndbox['ymin']))
                df_dict['xmax'].append(int(bndbox['xmax']))
                df_dict['ymax'].append(int(bndbox['ymax']))
                df_dict['pose'].append(obj['pose'])
                df_dict['truncated'].append(int(obj['truncated']))
                df_dict['difficult'].append(int(obj['difficult']))

        elif isinstance(objects, dict):
            df_dict['file_name'].append(xml_dict['annotation']['filename'])
            df_dict['file_type'].append(file_type)
            df_dict['object_length'].append(len(objects))
            df_dict['object'].append(objects['name'])

            bndbox = objects['bndbox']
            df_dict['xmin'].append(int(bndbox['xmin']))
            df_dict['ymin'].append(int(bndbox['ymin']))
            df_dict['xmax'].append(int(bndbox['xmax']))
            df_dict['ymax'].append(int(bndbox['ymax']))
            df_dict['pose'].append(objects['pose'])
            df_dict['truncated'].append(int(objects['truncated']))
            df_dict['difficult'].append(int(objects['difficult']))

In [ ]:
for file_name in samples['train']:
    parse_xml(annotation_path, file_name, 'train')

for file_name in samples['test']:
    parse_xml(annotation_path, file_name, 'test')

for file_name in samples['validation']:
    parse_xml(annotation_path, file_name, 'validation')

In [ ]:
with open('dataframe_dict.pickle','wb') as fw:
    pickle.dump(df_dict, fw)

In [ ]:
with open('dataframe_dict.pickle', 'rb') as fr:
    dataframe_dict = pickle.load(fr)

In [ ]:
df = pd.DataFrame.from_dict(dataframe_dict)
df.to_csv('df.csv', index=False)